In [195]:
import pandas as pd

In [196]:
df = pd.read_csv('./data/dataset.csv')
df

,entry,R1-,R2,organocatalyst,organocatalyst(mol%),solvent,under,temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),yield
0,1,[*]C1=CC=CC=C1,H,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,toluene,O2,90,24.0,0.0,0.0,81.0
1,2,[*]C1=CC=CC=C1,H,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,toluene,O2,90,12.0,0.0,0.0,14.0
2,3,[*]C1=CC=CC=C1,H,OC1=CC(C)=CC=C1C(O)=O,5.0,toluene,O2,90,24.0,0.0,0.0,74.0
3,4,[*]C1=CC=CC=C1,H,OC1=CC(C)=CC=C1C(O)=O,5.0,toluene,O2,90,12.0,0.0,0.0,15.0
4,5,[*]C1=CC=CC=C1,H,OC1=CC(OC)=CC=C1C(O)=O,5.0,toluene,O2,90,24.0,0.0,0.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...
201,2l-c,[*]C1=CC=CC=C1,Ph,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,72.0,0.0,0.0,92.0
202,2m,[*]C1=CSC=C1,C,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,48.0,0.0,0.0,83.0
203,2n,NaN,NaN,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,48.0,0.0,0.0,53.0
204,2o,NaN,NaN,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,48.0,0.0,0.0,25.0


# 前処理
1. 欠損値削除(R1-,R2,organocatalyst,yield)
2. under = Ar削除(1個)
3. R2 = Ph削除(3個)
4. 触媒促進剤使ってるデータも削除
5. rtを23度に置換
6. カテゴリ値をダミー変数に変換

In [197]:
# 基質、触媒、収率で欠損がある場合は削除
df.dropna(subset=['R1-', 'R2', 'organocatalyst', 'yield'], inplace=True)

In [198]:
print('欠損値合計', df.isnull().sum())
na_row = df.isnull().any(axis=1)
df.loc[na_row, :]

欠損値合計 entry                   0
R1-                     0
R2                      0
organocatalyst          0
organocatalyst(mol%)    0
solvent                 0
under                   0
temp(℃)                 0
time(h)                 0
Cu(OAc)2(mol%)          0
AcOH(mol%)              0
yield                   0
dtype: int64


,entry,R1-,R2,organocatalyst,organocatalyst(mol%),solvent,under,temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),yield


In [200]:
df.head()

,entry,R1-,R2,organocatalyst,organocatalyst(mol%),solvent,under,temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),yield
0,1,[*]C1=CC=CC=C1,H,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,toluene,O2,90,24.0,0.0,0.0,81.0
1,2,[*]C1=CC=CC=C1,H,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,toluene,O2,90,12.0,0.0,0.0,14.0
2,3,[*]C1=CC=CC=C1,H,OC1=CC(C)=CC=C1C(O)=O,5.0,toluene,O2,90,24.0,0.0,0.0,74.0
3,4,[*]C1=CC=CC=C1,H,OC1=CC(C)=CC=C1C(O)=O,5.0,toluene,O2,90,12.0,0.0,0.0,15.0
4,5,[*]C1=CC=CC=C1,H,OC1=CC(OC)=CC=C1C(O)=O,5.0,toluene,O2,90,24.0,0.0,0.0,90.0


In [191]:
# organocatalyst = '-'のデータ削除
df = df[df['organocatalyst'] != '-']
df['organocatalyst'] == '-'

0      False
1      False
2      False
3      False
4      False
       ...  
199    False
200    False
201    False
202    False
205    False
Name: organocatalyst, Length: 153, dtype: bool

In [192]:
# Arを削除(1 data)
df = df[df['under'] != 'Ar']
df['under'].value_counts()

O2     108
air     44
Name: under, dtype: int64

In [193]:
# R2 = Ph削除(3 data)
df = df[df['R2'] != 'Ph']
df['R2'].value_counts()

H    97
C    52
Name: R2, dtype: int64

In [194]:
# 触媒促進剤データ削除
df = df[df['Cu(OAc)2(mol%)'] == 0]
df = df[df['AcOH(mol%)'] == 0]
print(df.columns)
print(df.shape)

Index(['entry', 'R1-', 'R2', 'organocatalyst', 'organocatalyst(mol%)',
       'solvent', 'under', 'temp(℃)', 'time(h)', 'Cu(OAc)2(mol%)',
       'AcOH(mol%)', 'yield'],
      dtype='object')
(121, 12)


In [157]:
# 触媒カラム削除
df.drop(columns=['Cu(OAc)2(mol%)','AcOH(mol%)'],inplace=True)

df.columns

Index(['entry', 'R1-', 'R2', 'organocatalyst', 'organocatalyst(mol%)',
       'solvent', 'under', 'temp(℃)', 'time(h)', 'yield'],
      dtype='object')

In [158]:
df.reset_index(inplace=True,drop=True)

In [159]:
# rtを23度に置換
df['temp(℃)'][df['temp(℃)'] == 'rt'] = "23"
print(df['temp(℃)'].value_counts())

23    64
90    48
80     7
60     2
Name: temp(℃), dtype: int64


/var/folders/12/6r_lrg9d6kx949nw6gcq9ksh0000gn/T/ipykernel_92445/2448397563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp(℃)'][df['temp(℃)'] == 'rt'] = "23"


In [160]:
# カテゴリ値をダミー変数に変換
df = pd.get_dummies(df, columns=['R2','under','solvent'])
df.head(5)

,entry,R1-,organocatalyst,organocatalyst(mol%),temp(℃),time(h),yield,R2_C,R2_H,under_O2,under_air,solvent_CH3CN,solvent_DMF,solvent_MeCN,solvent_MeOH,solvent_toluene
0,1,[*]C1=CC=CC=C1,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,90,24.0,81.0,0,1,1,0,0,0,0,0,1
1,2,[*]C1=CC=CC=C1,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,90,12.0,14.0,0,1,1,0,0,0,0,0,1
2,3,[*]C1=CC=CC=C1,OC1=CC(C)=CC=C1C(O)=O,5.0,90,24.0,74.0,0,1,1,0,0,0,0,0,1
3,4,[*]C1=CC=CC=C1,OC1=CC(C)=CC=C1C(O)=O,5.0,90,12.0,15.0,0,1,1,0,0,0,0,0,1
4,5,[*]C1=CC=CC=C1,OC1=CC(OC)=CC=C1C(O)=O,5.0,90,24.0,90.0,0,1,1,0,0,0,0,0,1


In [108]:
# 重複確認
# check_columns = df.columns.drop(['entry', '収率(%)'])

# # 説明変数のみでみた時の重複削除
# _df = df.drop(columns='entry').drop_duplicates(subset=check_columns).reset_index(drop=True)
# X = _df.drop(columns=['収率(%)'])
# y = _df['収率(%)']

In [161]:
# 重複確認
def isDuplicated(df):
    check_columns = df.columns.drop(['entry', 'yield'])
    return df.duplicated(subset=check_columns).value_counts()

In [162]:
isDuplicated(df)

False    121
dtype: int64

In [163]:
df.to_csv('./data/origin.csv')

# スケーリング

In [164]:
# 説明変数をスケーリング
# 目的変数は必要ない。
# https://stats.stackexchange.com/questions/111467/is-it-necessary-to-scale-the-target-value-in-addition-to-scaling-features-for-re
from sklearn.preprocessing import StandardScaler
def scaling(df, scaling_columns):
    scaler = StandardScaler()
    scaler.fit(df[scaling_columns])
    df_scaled = pd.concat([df.drop(columns=scaling_columns), 
                               pd.DataFrame(scaler.transform(df[scaling_columns]),index=df.index , columns=scaling_columns)
                          ],axis=1, join='inner')
    print(df_scaled.shape)
    return df_scaled

In [165]:
scaling_columns = ['organocatalyst(mol%)','temp(℃)','time(h)']
df_scaled = scaling(df, scaling_columns)

(121, 16)


In [166]:
df_scaled.to_csv('./data/origin_scaled.csv')

# 記述子
https://qiita.com/ottantachinque/items/d7332bcf67b5844f5be3
### morganフィンガープリントの作成（Extended Connectivity Fingerprint）
子からある距離にある部分構造を数え上げていく，「circular substructures」と呼ばれるタイプのフィンガープリントです．radiusの値で距離を設定します．いわゆるECFP（Extended Connectivity Fingerprint）フィンガープリントに相当するものですが，探索距離の定義が異なる点に注意が必要です．
### morgan featureフィンガープリントの作成(Functional Connectivity Fingerprint）
FCFP（Functional Connectivity Fingerprint）様のフィンガープリントを用いたい場合にはuseFeatures=Trueを設定します．

### MACCS
全部で166の部分構造についての有無を調べ上げたもので，RDKit内の情報保持のために1ビット使っているため全部で167ビットのフィンガープリントになります．部分構造を有する場合は1が，ない場合は0が格納されています．

### RDkit記述子
一定の結合数に相当する原子と結合種類を格納する方法で，事前に部分構造を用意する必要がないことから，より柔軟に分子構造を表現することが可能です．

### MinHashフィンガープリント
「A probabilistic molecular fingerprint for big data settings」という論文で報告されている，ECFPと同様にCircular型のフィンガープリントになります

### Avalonフィンガープリント

### アトムペアフィンガープリントとトポロジカル二面角フィンガープリント
原子のアトムタイプとその最小結合距離を記録していきます．
アトムタイプには
原子の種類
結合する重原子の数
π電子数
https://future-chem.com/ap-dp-fingerprint/

### ドナーアクセプターペアフィンガープリント
ドナーアクセプターペアフィンガープリントでは「アトムタイプの情報」について

水素結合供与体
水素結合受容体
といった一般性のある「化学的性質」を情報として用いているフィンガープリントです

### mordred記述子の作成
https://github.com/mordred-descriptor


MolLogP : Crippenらによる原子ベースのLogP指標(LogPは水/オクタノール分配係数）
MolWt : 分子量
NumHAcceptors : 水素結合アクセプターの数
NumHDonors : 水素結合ドナーの数
NumHeteroatoms : へテロ元素の数
NumRotatableBonds : 回転可能な結合数
FractionCSP3 : 全炭素数におけるsp3炭素の割合
TPSA : トポロジカル極性表面積


In [167]:
BIT = 1024

In [168]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Avalon import pyAvalonTools
from mordred import Calculator, descriptors
from rdkit.Chem.AtomPairs.Sheridan import GetBPFingerprint
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem import rdMHFPFingerprint
encoder = rdMHFPFingerprint.MHFPEncoder()

def toFinger(df, columns, finger_method):
    df_copy = df.drop(columns=columns)
    for column in columns:
        fingerprints = toFingerFromSmiles(df[column],finger_method)
        if finger_method in ['mordred_2d','mordred_3d']:
            fingerprints_df = fingerprints
        else:
            column_names = list(map(lambda x: str(x)+'_'+column, range(len(fingerprints[0]))))
            fingerprints_df =  pd.DataFrame(fingerprints,index=df.index, columns=column_names)
        
        # scaling必要
        if finger_method in ['mordred_2d','mordred_3d', 'minhash']:
            fingerprints_df = scaling(fingerprints_df.round(5), fingerprints_df.columns)
            
        df_copy = pd.merge(df_copy, fingerprints_df, left_index=True, right_index=True)
    return df_copy

def toFingerFromSmiles(series, method):
    mols = []
    for smile in series:
        if smile in ['-', 0]: smile = '' 
        mols.append(Chem.MolFromSmiles(smile))

    if (method == 'mordred_2d'):
        # scaling必要
        calc_2D = Calculator(descriptors, ignore_3D=True) #2D記述子
        fingerprints = calc_2D.pandas(mols,quiet=False)
        return fingerprints
    elif (method == 'mordred_3d'):
        # scaling必要
        calc_3D = Calculator(descriptors, ignore_3D=False) #3D記述子
        fingerprints = calc_3D.pandas(mols,quiet=False)
        return fingerprints
    else:
        fingerprints = []
        for mol_idx, mol in enumerate(mols):
            try:
                # listに直してる。
                if (method == 'morgan'):
                    fingerprint = [x for x in AllChem.GetMorganFingerprintAsBitVect(mol,2,BIT)]
                elif (method == 'morgan_feature'):
                    fingerprint = [x for x in AllChem.GetMorganFingerprintAsBitVect(mol,2,BIT,useFeatures=True)]
                elif (method == 'maccs'):
                    fingerprint = [x for x in AllChem.GetMACCSKeysFingerprint(mol)]
                elif (method == 'rdkit'):
                    fingerprint = [x for x in Chem.RDKFingerprint(mol)]
                elif (method == 'minhash'):
                    # scaling必要
                    fingerprint = [x for x in encoder.EncodeMol(mol)]
                elif (method == 'avalon'):
                    fingerprint = [x for x in pyAvalonTools.GetAvalonFP(mol)]
                elif (method == 'atom'):
                    fingerprint = [x for x in Pairs.GetAtomPairFingerprintAsBitVect(mol)]
                elif (method == 'donor'):
                    print('donor')
                    fingerprint = [x for x in GetBPFingerprint(mol)]
                else:
                    print('method error')
                fingerprints.append(fingerprint)
            except Exception as e:
                print(e)
                print("Error", mol_idx)
                break
        return fingerprints

In [169]:
finger_methods = [
    'morgan',
    'morgan_feature',
    'maccs',
    'rdkit',
    'minhash',
    'avalon',
    #'atom', # 重い
    #'donor', # 重い
    'mordred_2d',
    'mordred_3d',
]

# 記述子変換 ⇒ 分散が0 or 80%異常が同じ値の情報を削除
説明しきれていないため共線性の恐れがある
VarianceThresholdで特徴選択

In [177]:
from sklearn.feature_selection import VarianceThreshold
def removeNoImpactData(df):
    X = df.drop(columns=['entry','yield'])
    select = VarianceThreshold()
    X_new = select.fit_transform(X)
    df_new = pd.concat([
        df['yield'],
        pd.DataFrame(X_new, columns=np.array(X.columns)[select.get_support()==True])
    ],axis=1)
    print('before_shape: ', df.shape)
    print('new_shape: ', df_new.shape)
    return df_new

# finger print変換
- method1 全てのデータを使用

In [178]:
# 1. 全てのデータを使用
finger_columns = ['R1-', 'organocatalyst']
for finger_method in finger_methods:
    print(finger_method)
    df_method1 = toFinger(df_scaled, finger_columns,finger_method)
    removeNoImpactData(df_method1).to_csv(f'./data/method1/{finger_method}.csv')

morgan
before_shape:  (121, 2062)
new_shape:  (121, 363)
morgan_feature
before_shape:  (121, 2062)
new_shape:  (121, 232)
maccs
before_shape:  (121, 348)
new_shape:  (121, 175)
rdkit
before_shape:  (121, 4110)
new_shape:  (121, 2642)
minhash
(121, 2048)
(121, 2048)
before_shape:  (121, 4110)
new_shape:  (121, 4103)
avalon
before_shape:  (121, 1038)
new_shape:  (121, 567)
mordred_2d


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [00:16<00:00,  7.37it/s]


(121, 1613)


/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:990: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [00:41<00:00,  2.91it/s]
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in divide
  updated_

(121, 1613)
before_shape:  (121, 3240)
new_shape:  (121, 1726)
mordred_3d


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [00:28<00:00,  4.30it/s]


(121, 1826)


/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:990: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [00:54<00:00,  2.24it/s]
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in divide
  updated_

(121, 1826)
before_shape:  (121, 3666)
new_shape:  (121, 1726)


# method2 
- R1 [*]C1=CC=CC=C1 
- R2_H = 1
- solvent toluene

In [179]:
df = pd.read_csv('./data/origin.csv', index_col=0)

In [180]:
for column in df.columns:
    print(df[column].value_counts())


1       7
3       7
2       7
4       6
6       6
5       5
7       5
13      4
12      4
9       4
8       4
10      3
11      3
2a      3
2b      3
2c      3
2d      3
2e      3
2f      3
2g      3
2j      2
2k      2
2m      2
2i      2
2h      2
3f      1
2h-c    1
3q      1
3o      1
3n      1
3m      1
3l      1
3k      1
3j      1
3i      1
3h      1
3g      1
3e      1
2l      1
3d      1
24      1
22      1
21      1
20      1
19      1
14      1
1p      1
1o      1
1n      1
2p      1
Name: entry, dtype: int64
[*]C1=CC=CC=C1                  49
[*]C1=CC=C(OC)C=C1              15
[*]C1=CC=C(C)C=C1                7
[*]C1=CC=C(Cl)C=C1               4
[*]C1=CC(OCO2)=C2C=C1            3
[*]C1=CC=C(F)C=C1                3
[*]C1=CC=CO1                     3
[*]C1=CC(Cl)=CC=C1               2
[*]C1=CC=C([N+]([O-])=O)C=C1     2
[*]C1=CC=CC(Cl)=C1               2
[*]C1=C(C)C=CC=C1                2
[*]C1=CC=CS1                     2
[*]C1=C(C=CC=C2)C2=CC=C1         2
[*]C1=CC=C(C(F)(F)F

In [181]:
df_ = df[df['R1-'] == '[*]C1=CC=CC=C1']
df_ = df_[df_['R2_H'] == 1]
df_ = df_[df_['solvent_toluene'] == 1]
df_.shape

(27, 16)

In [182]:
df_dropped = df_.drop(columns=['R1-', 'R2_C', 'R2_H', 'solvent_CH3CN', 'solvent_DMF', 'solvent_MeCN', 'solvent_MeOH', 'solvent_toluene', 'under_O2', 'under_air','organocatalyst(mol%)', 'temp(℃)'])




In [183]:
scaling_columns = ['time(h)']
df_dropped_scaled = scaling(df_dropped, scaling_columns).reset_index(drop=True)

(27, 4)


In [184]:
# 2. 限りなく除去
finger_columns = ['organocatalyst']
for finger_method in finger_methods:
    print(finger_method)
    df_method2 = toFinger(df_dropped_scaled, finger_columns,finger_method)
    removeNoImpactData(df_method2).to_csv(f'./data/method2/{finger_method}.csv')

morgan
before_shape:  (27, 1027)
new_shape:  (27, 113)
morgan_feature
before_shape:  (27, 1027)
new_shape:  (27, 84)
maccs
before_shape:  (27, 170)
new_shape:  (27, 73)
rdkit
before_shape:  (27, 2051)
new_shape:  (27, 1082)
minhash
(27, 2048)
before_shape:  (27, 2051)
new_shape:  (27, 2050)
avalon
before_shape:  (27, 515)
new_shape:  (27, 266)
mordred_2d


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:11<00:00,  2.38it/s]
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:990: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/feature_selection/_variance_threshold.py:104: RuntimeWarning: Degrees of freedom <= 

(27, 1613)
before_shape:  (27, 1616)
new_shape:  (27, 1148)
mordred_3d


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:08<00:00,  3.30it/s]


(27, 1826)
before_shape:  (27, 1829)
new_shape:  (27, 1148)


/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:990: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/feature_selection/_variance_threshold.py:104: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/feature_selection/_variance_threshold.py:112: RuntimeWarning: All-NaN

# method3
- solvent MeCN, CHCN